## Домашнее задание

Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [29]:
# Загрузка необходимых библиотек

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, log_loss

import dill

In [2]:
# Загрузка данных

df = pd.read_csv('./WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Проверка

df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [3]:
#Ваш код здесь

df.groupby(['gender']).size()

gender
Female    3488
Male      3555
dtype: int64

##### 2. Какое количество уникальных значений у поля InternetService?

In [4]:
#Ваш код здесь

len(df['InternetService'].unique())

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [5]:
#Ваш код здесь

df[df['TotalCharges'] == ' '] = 0
pd.to_numeric(df['TotalCharges'], downcast = 'float').describe()

count    7043.000000
mean     2279.734375
std      2266.794434
min         0.000000
25%       398.549988
50%      1394.550049
75%      3786.599976
max      8684.799805
Name: TotalCharges, dtype: float64

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

**Ответ:** для объектов нельзя посчитать числовые величины сразу

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [6]:
#Ваш код здесь

df.loc[df['PhoneService'] == 'Yes', 'PhoneService'] = 1
df.loc[df['PhoneService'] == 'No', 'PhoneService'] = 0

# Проверка

df['PhoneService']

0       0
1       1
2       1
3       0
4       1
       ..
7038    1
7039    1
7040    0
7041    1
7042    1
Name: PhoneService, Length: 7043, dtype: object

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [7]:
#Ваш код здесь

df.loc[df['TotalCharges'] == ' ', 'TotalCharges'] = np.nan
df['TotalCharges'] = df['TotalCharges'].astype(np.float32).fillna(0)
df['TotalCharges'].describe()

count    7043.000000
mean     2279.734375
std      2266.794434
min         0.000000
25%       398.549988
50%      1394.550049
75%      3786.599976
max      8684.799805
Name: TotalCharges, dtype: float64

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [8]:
#Ваш код здесь

df.loc[df['Churn'] == 'Yes', 'Churn'] = 1
df.loc[df['Churn'] == 'No', 'Churn'] = 0
df['Churn'] = df['Churn'].astype(np.int0)

# Проверка

df['Churn']

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [9]:
#Ваш код здесь

columns = ['StreamingMovies', 'StreamingTV', 'TechSupport']

for column in columns:
    
    df[column] = df[column].map(lambda x: 1 if x == 'Yes' or x == 1 else 0)
    
# Проверка

df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,0,No phone service,DSL,No,...,No,0,0,0,Month-to-month,Yes,Electronic check,29.85,29.850000,0
1,5575-GNVDE,Male,0,No,No,34,1,No,DSL,Yes,...,Yes,0,0,0,One year,No,Mailed check,56.95,1889.500000,0
2,3668-QPYBK,Male,0,No,No,2,1,No,DSL,Yes,...,No,0,0,0,Month-to-month,Yes,Mailed check,53.85,108.150002,1
3,7795-CFOCW,Male,0,No,No,45,0,No phone service,DSL,Yes,...,Yes,1,0,0,One year,No,Bank transfer (automatic),42.30,1840.750000,0
4,9237-HQITU,Female,0,No,No,2,1,No,Fiber optic,No,...,No,0,0,0,Month-to-month,Yes,Electronic check,70.70,151.649994,1
5,9305-CDSKC,Female,0,No,No,8,1,Yes,Fiber optic,No,...,Yes,0,1,1,Month-to-month,Yes,Electronic check,99.65,820.500000,1
6,1452-KIOVK,Male,0,No,Yes,22,1,Yes,Fiber optic,No,...,No,0,1,0,Month-to-month,Yes,Credit card (automatic),89.10,1949.400024,0
7,6713-OKOMC,Female,0,No,No,10,0,No phone service,DSL,Yes,...,No,0,0,0,Month-to-month,No,Mailed check,29.75,301.899994,0
8,7892-POOKP,Female,0,Yes,No,28,1,Yes,Fiber optic,No,...,Yes,1,1,1,Month-to-month,Yes,Electronic check,104.80,3046.050049,1
9,6388-TABGU,Male,0,No,Yes,62,1,No,DSL,Yes,...,No,0,0,0,One year,No,Bank transfer (automatic),56.15,3487.949951,0


##### 8. Заполните пропуски в поле PhoneService значением 0

In [10]:
#Ваш код здесь

df['PhoneService'] = pd.to_numeric(df['PhoneService'], errors = 'coerce', downcast = 'float')
df.loc[df['PhoneService'].isna(), 'PhoneService'] = 0

# Проверка

df['PhoneService'].unique()

array([0., 1.], dtype=float32)

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [11]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']

# Ваш код здесь

df = df[columns]

# Проверка

df.head(10)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0.0,29.850000,0,0,0,0
1,Male,34,1.0,1889.500000,0,0,0,0
2,Male,2,1.0,108.150002,0,0,0,1
3,Male,45,0.0,1840.750000,0,0,1,0
4,Female,2,1.0,151.649994,0,0,0,1
5,Female,8,1.0,820.500000,1,1,0,1
6,Male,22,1.0,1949.400024,0,1,0,0
7,Female,10,0.0,301.899994,0,0,0,0
8,Female,28,1.0,3046.050049,1,1,1,1
9,Male,62,1.0,3487.949951,0,0,0,0


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [12]:
features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']

target = 'Churn'

#Ваш код здесь

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], random_state = 42)

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [13]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [14]:
# Pipeline

gender = Pipeline([
                ('selector', ColumnSelector(key = 'gender')),
                ('ohe', OHEEncoder(key = 'gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [15]:
#Ваш код здесь

gender.fit_transform(X_train)

,gender_0,gender_Female,gender_Male
6607,0,0,1
2598,0,1,0
2345,0,1,0
4093,0,1,0
693,0,1,0
...,...,...,...
3772,0,0,1
5191,0,1,0
5226,0,0,1
5390,0,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [17]:
tenure =  Pipeline([
                ('selector', NumberSelector(key = 'tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [18]:
#Ваш код здесь

total_charges =  Pipeline([
                ('selector', NumberSelector(key = 'TotalCharges')),
                ('standard', StandardScaler())
            ])

Объединение всех "кубиков" очень легко сделать таким образом

In [20]:
number_features = Pipeline([
                ('selector', ColumnSelector(key = ['PhoneService',
                                                   'StreamingMovies', 'StreamingTV', 
                                                   'TechSupport']))
            ])

In [22]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', total_charges),
                      ('continuos_features', number_features),
                      ('gender', gender)])

feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [24]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [25]:
#Ваш код здесь

pipeline.predict(X_test)

array([1, 0, 0, ..., 0, 1, 0])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [27]:
#Ваш код здесь

roc_auc_score(y_true = y_test, y_score = pipeline.predict_proba(X_test)[:, 1])

0.8062843808115582

In [28]:
# Log loss

log_loss(y_true = y_test, y_pred = pipeline.predict_proba(X_test))

0.6921688496943421

### Сохраним наш пайплайн

In [30]:
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)